![NYC Skyline](img/nyc.jpg)

Welcome to New York City, one of the most-visited cities in the world. There are many Airbnb listings in New York City to meet the high demand for temporary lodging for travelers, which can be anywhere between a few nights to many months. In this project, you will take a closer look at the New York Airbnb market by combining data from multiple file types like `.csv`, `.tsv`, and `.xlsx` (Excel files).

Recall that **CSV**, **TSV**, and **Excel** files are three common formats for storing data. 
Three files containing data on 2019 Airbnb listings are available to you:

**data/airbnb_price.csv**
This is a CSV file containing data on Airbnb listing prices and locations.
- **`listing_id`**: unique identifier of listing
- **`price`**: nightly listing price in USD
- **`nbhood_full`**: name of borough and neighborhood where listing is located

**data/airbnb_room_type.xlsx**
This is an Excel file containing data on Airbnb listing descriptions and room types.
- **`listing_id`**: unique identifier of listing
- **`description`**: listing description
- **`room_type`**: Airbnb has three types of rooms: shared rooms, private rooms, and entire homes/apartments

**data/airbnb_last_review.tsv**
This is a TSV file containing data on Airbnb host names and review dates.
- **`listing_id`**: unique identifier of listing
- **`host_name`**: name of listing host
- **`last_review`**: date when the listing was last reviewed


In [52]:
# We've loaded the necessary packages for you in the first cell. Please feel free to add as many cells as you like!
suppressMessages(library(dplyr)) # This line is required to check your answer correctly
options(readr.show_types = FALSE) # This line is required to check your answer correctly
library(readr)
library(readxl)
library(stringr)

In [53]:
# Read the datasets from the three files in the data folder: airbnb_price.csv, airbnb_room_type.xlsx, airbnb_last_review.tsv.
airbnb_price <- read_csv("./data/airbnb_price.csv")
airbnb_room_type <- read_excel("./data/airbnb_room_type.xlsx")
airbnb_last_review <- read_tsv("./data/airbnb_last_review.tsv")

# joining all the dataframes together
listings <- airbnb_price %>%
    inner_join(airbnb_room_type, by="listing_id") %>%
    inner_join(airbnb_last_review, by="listing_id")

head(listings, 5)

Rows: 25209 Columns: 3
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (2): price, nbhood_full
dbl (1): listing_id

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 25209 Columns: 3
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr (2): host_name, last_review
dbl (1): listing_id

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


listing_id,price,nbhood_full,description,room_type,host_name,last_review
<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
2595,225 dollars,"Manhattan, Midtown",Skylit Midtown Castle,Entire home/apt,Jennifer,May 21 2019
3831,89 dollars,"Brooklyn, Clinton Hill",Cozy Entire Floor of Brownstone,Entire home/apt,LisaRoxanne,July 05 2019
5099,200 dollars,"Manhattan, Murray Hill",Large Cozy 1 BR Apartment In Midtown East,Entire home/apt,Chris,June 22 2019
5178,79 dollars,"Manhattan, Hell's Kitchen",Large Furnished Room Near B'way,private room,Shunichi,June 24 2019
5238,150 dollars,"Manhattan, Chinatown",Cute & Cozy Lower East Side 1 bdrm,Entire home/apt,Ben,June 09 2019


In [54]:
# checking the data type of the dataframe
glimpse(listings)

Rows: 25,209
Columns: 7
$ listing_id  <dbl> 2595, 3831, 5099, 5178, 5238, 5295, 5441, 5803, 6021, 6848…
$ price       <chr> "225 dollars", "89 dollars", "200 dollars", "79 dollars", …
$ nbhood_full <chr> "Manhattan, Midtown", "Brooklyn, Clinton Hill", "Manhattan…
$ description <chr> "Skylit Midtown Castle", "Cozy Entire Floor of Brownstone"…
$ room_type   <chr> "Entire home/apt", "Entire home/apt", "Entire home/apt", "…
$ host_name   <chr> "Jennifer", "LisaRoxanne", "Chris", "Shunichi", "Ben", "Le…
$ last_review <chr> "May 21 2019", "July 05 2019", "June 22 2019", "June 24 20…


# What are the dates of the earliest and most recent reviews?

In [55]:
# Convert the dates into datetime type
listings = listings %>%
    mutate(last_review_date = as.Date(last_review, format= "%B %d %Y"))
# Get dates
review_dates <- listings %>%
    summarize(first_reviewed = min(last_review_date),
              last_reviewed = max(last_review_date))
review_dates
print(paste("The first reviewed date is", review_dates$first_reviewed))
print(paste("The last reviewed date is", review_dates$last_reviewed))

first_reviewed,last_reviewed
<date>,<date>
2019-01-01,2019-07-09


[1] "The first reviewed date is 2019-01-01"
[1] "The last reviewed date is 2019-07-09"


# How many of the listings are private rooms? 

In [56]:
# Get the frequency of the room_tye category
listings %>%
    count(room_type)

room_type,n
<chr>,<int>
ENTIRE HOME/APT,2143
Entire home/apt,8458
PRIVATE ROOM,1867
Private room,7241
SHARED ROOM,97
Shared room,380
entire home/apt,2665
private room,2248
shared room,110


In [57]:
# The table shows that the room_type column contain the same room type but written in different cases.
# Converting room types to a standard lower case
listings = listings %>%
    mutate(room_type = str_to_lower(room_type))

# Then get the frequency of the room_tye category again
listings %>%
    count(room_type)

room_type,n
<chr>,<int>
entire home/apt,13266
private room,11356
shared room,587


In [58]:
# Get the total number of private rooms
private_room_count <- listings %>%
    count(room_type) %>%
    filter(room_type == "private room")

nb_private_rooms <- private_room_count$n
print(paste("The total number of private rooms is", nb_private_rooms))

[1] "The total number of private rooms is 11356"


# What is the average listing price?

In [59]:
# The listing price variable is a string that contains the word "dollars", but it needs to be numeric. E.g. "225 dollars".
# Remove the word dollars and change the data tpe of price from string to numeric
listings = listings %>%
    mutate(price = str_remove(price, " dollars") %>%
           as.numeric())

glimpse(listings)

Rows: 25,209
Columns: 8
$ listing_id       <dbl> 2595, 3831, 5099, 5178, 5238, 5295, 5441, 5803, 6021,…
$ price            <dbl> 225, 89, 200, 79, 150, 135, 85, 89, 85, 140, 215, 140…
$ nbhood_full      <chr> "Manhattan, Midtown", "Brooklyn, Clinton Hill", "Manh…
$ description      <chr> "Skylit Midtown Castle", "Cozy Entire Floor of Browns…
$ room_type        <chr> "entire home/apt", "entire home/apt", "entire home/ap…
$ host_name        <chr> "Jennifer", "LisaRoxanne", "Chris", "Shunichi", "Ben"…
$ last_review      <chr> "May 21 2019", "July 05 2019", "June 22 2019", "June …
$ last_review_date <date> 2019-05-21, 2019-07-05, 2019-06-22, 2019-06-24, 2019…


In [60]:
# Calculate the average listing price
avg_price <- listings %>%
    summarize(avg_price = mean(price)) %>%
    as.numeric()

print(paste0("The average listing price is $", round(avg_price, digits = 2))) # the paste0 function removes the space between the $ sign and the number

[1] "The average listing price is $141.78"


# Create a new tibble.

In [61]:
# Combine the new variables into one tibble called review_dates with four columns in the following order: first_reviewed, last_reviewed, nb_private_rooms, and avg_price. The tibble should only contain one row of values.
review_dates <- tibble(
  first_reviewed = c(review_dates$first_reviewed),
  last_reviewed = c(review_dates$last_reviewed),
  nb_private_rooms = c(nb_private_rooms),
  avg_price = c(avg_price)
)

print(review_dates)

# A tibble: 1 × 4
  first_reviewed last_reviewed nb_private_rooms avg_price
  <date>         <date>                   <int>     <dbl>
1 2019-01-01     2019-07-09               11356      142.
